导入必要的Python库：

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import pickle
import time
from Data import Data

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

设置__可视化参数__：

In [ ]:
mpl.rcParams['figure.figsize'] = (20, 6)
mpl.rcParams['axes.grid'] = False

初始化__随机种子__：

In [ ]:
tf.random.set_seed(2020)

设置默认__浮点数类型__：

In [ ]:
tf.keras.backend.set_floatx('float64')

载入数据：

In [ ]:
# Alibaba dataset
# data = dict()
# for filename in os.listdir('.'):
#     if filename.endswith('.pickle'):
#         with open(filename, 'rb') as f:
#             data[filename[filename.rindex('_') + 1:filename.index('.')]] = pickle.load(f)[3000:]
# data = pd.DataFrame(data)
# data.head(5)

# Bustracker Dataset
data = pd.read_csv('BusTrackerData/1.csv',names=['date', 'freq'])
# data = pd.read_csv('SDSS/DTW/149.csv',names=['date', 'freq'])
data = data.dropna(axis=0,how='any')
data['date'] = pd.to_datetime(data['date'])
# df.reset_index(drop=True,inplace=True)
data = data.set_index('date')

# SDSS Dataset
# data = pd.read_csv('SDSS/DTW149.csv',names=['date', 'freq'])
# data = data.dropna(axis=0,how='any')
# data['date'] = pd.to_datetime(data['date'])
# # df.reset_index(drop=True,inplace=True)
# data = data.set_index('date')

    
data.head(5)

数据可视化：

In [ ]:
data.plot(subplots=True)
plt.show()

In [ ]:
# TRAIN_SPLIT = 800
TRAIN_SPLIT = 7000  # Bustracker
# TRAIN_SPLIT = 4000  # SDSS

HISTORY_SIZE = 30
# TARGET_SIZE = 0
bus_series = data['freq'].values
print(len(bus_series))
print(bus_series)

In [ ]:
from statsmodels.tsa.api import SARIMAX

test_length = len(bus_series) - TRAIN_SPLIT
horizon = 5

pred_list = []
order_parm = (2,1,2)
y_true_list = bus_series[TRAIN_SPLIT+horizon:]
for i in range(test_length - horizon):
    tempList = bus_series[:TRAIN_SPLIT+i]
    for j in range(horizon+1):
        fit = SARIMAX(tempList, order=order_parm).fit(disp=0)
        forecast = fit.forecast()[0]
        tempList = np.append(tempList, forecast)
    pred_list.append(forecast)

In [ ]:
print(len(pred_list))
print(len(y_true_list))

In [ ]:
mse = tf.keras.losses.MeanSquaredError()

final_loss = mse(np.array(pred_list), np.array(y_true_list)).numpy()
print(final_loss)

In [ ]:
resultFrame = dict()
resultFrame['True'] = y_true_list
resultFrame['Arima'] = pred_list
frame = pd.DataFrame(resultFrame)
frame.to_csv('Model/BusTracker/Arima' + str(order_parm) + '-' + str(horizon) + '.csv')